# CellRank 2: Unified Fate Mapping in Multiview Single-Cell Data

## Workshop Tutorial — Informatics Club

---

### What is CellRank 2?

**CellRank 2** is a framework for studying **cellular fate decisions** from single-cell data. It models cell-state transitions as a **Markov chain** on a cell-cell graph and identifies:

- **Initial states** (cells at the beginning of a process)
- **Terminal states / macrostates** (fate endpoints)
- **Fate probabilities** (likelihood each cell reaches each terminal state)
- **Driver genes** correlated with specific fate decisions

### Key Innovation: Modular Kernel Framework

CellRank 1 relied exclusively on **RNA velocity**. CellRank 2 introduces a **modular kernel framework** that can incorporate diverse sources of directional information:

| Kernel | Input | Use Case |
|--------|-------|----------|
| `VelocityKernel` | RNA velocity vectors | Standard scRNA-seq with splicing info |
| `PseudotimeKernel` | Pseudotime ordering | When a pseudotime is available |
| `RealTimeKernel` | Experimental time labels + optimal transport | Time-series experiments |
| `CytoTRACEKernel` | CytoTRACE scores (gene counts as proxy for potency) | When no velocity/time is available |

Kernels can be **combined** via weighted sums or products to integrate multiple signals.

### This Tutorial

This notebook follows the **CellRank 2 Protocol** (Weiler et al., Nature Protocols 2024) using three datasets:

1. **Bone marrow** (human hematopoiesis) — Procedures 1 & 2: CytoTRACEKernel + PseudotimeKernel
2. **Spermatogenesis** (mouse) — Procedure 3: VelocityKernel
3. **LARRY** (mouse hematopoiesis with lineage tracing) — Procedure 4: RealTimeKernel + optimal transport

---

## Part 0: Mathematical Background

### The Markov Chain Model

CellRank models cell-state dynamics as a **Markov chain** on a KNN graph of cells. Each cell is a node, and directed, weighted edges represent transition probabilities.

The **transition matrix** $T \in \mathbb{R}^{n \times n}$ is row-stochastic:

$$T_{ij} = P(X_{t+1} = j \mid X_t = i), \quad \sum_j T_{ij} = 1$$

### How Kernels Build the Transition Matrix

#### Velocity Kernel
For each cell $i$ with velocity vector $v_i$, the transition probability to neighbor $j$ is based on the **cosine similarity** between $v_i$ and the displacement vector $(x_j - x_i)$:

$$\tilde{T}_{ij} \propto \exp\left(\frac{\cos(v_i, x_j - x_i)}{\sigma}\right)$$

where $\sigma$ controls the softmax sharpness.

#### Pseudotime Kernel
Uses a **pseudotime** $\tau_i$ assigned to each cell. The kernel biases transitions toward increasing pseudotime:

$$\tilde{T}_{ij} \propto \begin{cases} \text{high} & \text{if } \tau_j > \tau_i \\ \text{low} & \text{if } \tau_j < \tau_i \end{cases}$$

#### RealTimeKernel (Optimal Transport)
Given cells at discrete time points $t_1, t_2, \ldots$, **optimal transport** (via the `moscot` package) computes a coupling matrix $\pi$ that maps cells at $t_k$ to cells at $t_{k+1}$:

$$\pi^* = \arg\min_\pi \sum_{i,j} c(x_i, x_j) \pi_{ij} + \varepsilon H(\pi)$$

where $c(x_i, x_j)$ is a cost (e.g., squared Euclidean distance in gene expression space), and $H(\pi)$ is the entropic regularization term.

#### CytoTRACE Kernel
Uses **CytoTRACE** scores as a proxy for developmental potential (based on the number of expressed genes). Biases transitions from high-potency to low-potency cells.

### Kernel Combination
Multiple kernels can be combined:

$$T_{\text{combined}} = \alpha \cdot T_{\text{kernel}_1} + (1 - \alpha) \cdot T_{\text{kernel}_2}$$

### GPCCA for Macrostate Identification

CellRank 2 uses **Generalized Perron Cluster Cluster Analysis (GPCCA)** on the transition matrix to identify **macrostates**. This works by computing the **Schur decomposition** of $T$:

$$T = Q R Q^{-1}$$

The leading Schur vectors reveal the coarse-grained structure. Terminal states are macrostates with high self-transition probability (absorbing).

### Absorption Probabilities (Fate Probabilities)

For each non-terminal cell, the probability of eventually reaching each terminal state:

$$\mathbf{a}_m = (I - T_{\text{transient}})^{-1} T_{\text{transient} \to m}$$

---

## Setup

```bash
conda env create -f environment.yml
conda activate cellrank2_workshop
```

### Download Datasets

Run these commands from the `sctalk3_cellrank2/` directory:

```bash
mkdir -p data/bone_marrow/processed data/spermatogenesis/processed data/larry/processed

# Bone marrow (human hematopoiesis)
wget https://figshare.com/ndownloader/files/53394941 -O data/bone_marrow/processed/adata.h5ad

# Spermatogenesis (mouse)
wget https://figshare.com/ndownloader/files/53395040 -O data/spermatogenesis/processed/adata.h5ad

# LARRY (mouse hematopoiesis with lineage tracing)
wget https://figshare.com/ndownloader/files/55781036 -O data/larry/processed/adata_subsetted_and_processed.zarr.zip
```

> **Note:** If `wget` is not available, use `curl -L -o <output> <url>` instead. If figshare blocks automated downloads, open the URLs in your browser to download manually.

In [ ]:
import cellrank as cr
import scanpy as sc
import scvelo as scv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

scv.settings.verbosity = 3
cr.settings.verbosity = 2
sc.settings.set_figure_params(frameon=False, dpi=100)

---

## Procedure 1: Identifying Initial & Terminal States (Bone Marrow — CytoTRACEKernel)

### About the Dataset

Human bone marrow scRNA-seq from **Setty et al. (2019)**. Hematopoietic stem cells (HSCs) differentiate into multiple blood cell lineages: monocytes, erythrocytes, megakaryocytes, etc.

We use the **CytoTRACEKernel** — based on the number of expressed genes as a proxy for developmental potential — to identify initial and terminal states without needing RNA velocity or pseudotime.

### Step 1: Load and Inspect Data

In [ ]:
adata_bm = sc.read_h5ad("data/bone_marrow/processed/adata.h5ad")
TERMINAL_STATES = ["Ery", "CLP", "cDC", "pDC", "Mono"]
STATE_TRANSITIONS = [
    ("HSC", "MEP"),
    ("MEP", "Ery"),
    ("HSC", "HMP"),
    ("HMP", "Mono"),
    ("HMP", "CLP"),
    ("HMP", "DCPre"),
    ("DCPre", "pDC"),
    ("DCPre", "cDC"),
]
adata_bm

In [ ]:
sc.pl.umap(adata_bm, color="celltype", legend_loc="right margin")

### Step 2: Compute CytoTRACEKernel

The CytoTRACEKernel uses the CytoTRACE score — based on the observation that the **number of expressed genes** decreases during differentiation — to compute transition probabilities. No RNA velocity or pseudotime needed.

In [ ]:
from cellrank.kernels import CytoTRACEKernel

ctk = CytoTRACEKernel(adata_bm)
ctk.compute_cytotrace(layer="MAGIC_imputed_data")
ctk.compute_transition_matrix(threshold_scheme="soft", nu=0.5)
print(ctk)

In [ ]:
# CytoTRACE stores its pseudotime in adata.obs — check what key it used
ct_key = [c for c in adata_bm.obs.columns if "ct_" in c or "cytotrace" in c.lower()]
print("CytoTRACE keys in obs:", ct_key)
sc.pl.umap(adata_bm, color=ct_key[0] if ct_key else "palantir_pseudotime", color_map="gnuplot", title="CytoTRACE Pseudotime")

In [ ]:
ctk.plot_projection(basis="umap", color="celltype")

### Step 3: Identify Macrostates with GPCCA

Use the GPCCA estimator to compute the Schur decomposition, identify macrostates, and classify terminal/initial states.

In [ ]:
from cellrank.estimators import GPCCA

g = GPCCA(ctk)
g.compute_schur(n_components=20)
g.plot_spectrum()

In [ ]:
g.plot_spectrum(real_only=True)

In [ ]:
ctk_n_states = None
for n_states in (15, 12, 10, 8, 6):
    try:
        g.compute_macrostates(n_states=n_states, cluster_key="celltype")
        ctk_n_states = n_states
        print(f"Using n_states={n_states} for CytoTRACEKernel GPCCA.")
        break
    except ValueError as e:
        print(f"n_states={n_states} failed: {e}")

if ctk_n_states is None:
    raise RuntimeError("Could not compute CytoTRACE macrostates with fallback schedule.")

g.plot_macrostates(which="all", basis="umap", legend_loc="right margin", s=100)

In [ ]:
g.plot_coarse_T()

### Step 4: Classify Terminal and Initial States

In [ ]:
g.predict_terminal_states()
g.plot_macrostates(which="terminal", basis="umap", legend_loc="right margin", s=100)

In [ ]:
g.predict_initial_states(allow_overlap=True)
g.plot_macrostates(which="initial", basis="umap", legend_loc="right margin", s=100)

### Step 5: Rename Terminal States

Rename the automatically identified terminal states to more meaningful names for downstream analysis.

In [ ]:
print("Current terminal states:", g.terminal_states.cat.categories.tolist())

In [ ]:
# Rename terminal states for clarity (adjust the mapping based on your macrostate output above)
# Example: g.rename_terminal_states({"Ery_2": "Erythroid", "Mono": "Monocyte"})
# For now, we'll keep the auto-detected names and proceed
print("Terminal states:", g.terminal_states.cat.categories.tolist())

---

## Procedure 2: Fate Probabilities & Driver Genes (Bone Marrow — PseudotimeKernel)

Now we switch to the **PseudotimeKernel** using Palantir diffusion pseudotime, and compute fate probabilities, driver genes, and gene expression trends.

### Step 6: Compute PseudotimeKernel

In [ ]:
from cellrank.kernels import PseudotimeKernel

pk = PseudotimeKernel(adata_bm, time_key="palantir_pseudotime")
pk.compute_transition_matrix(threshold_scheme="soft")
print(pk)

In [ ]:
pk.plot_projection(basis="umap", color="celltype")

### Step 7: Identify Terminal States with PseudotimeKernel

In [ ]:
g2 = GPCCA(pk)
g2.compute_schur(n_components=20)
g2.plot_spectrum(real_only=True)

In [ ]:
g2.compute_macrostates(n_states=6, cluster_key="celltype")
g2.predict_terminal_states()
g2.plot_macrostates(which="terminal", basis="umap", legend_loc="right margin", s=100)

### Step 8: Compute Fate Probabilities

For each cell, compute the probability of being absorbed into each terminal state — the **fate probabilities**.

In [ ]:
g2.compute_fate_probabilities()
print(g2.fate_probabilities)

In [ ]:
g2.plot_fate_probabilities(basis="umap", same_plot=False)

In [ ]:
cr.pl.circular_projection(adata_bm, keys=["celltype"], legend_loc="right")

### Step 9: Identify Driver Genes

Driver genes are genes whose expression is significantly **correlated with fate probabilities** toward a given terminal state.

In [ ]:
terminal_states = g2.terminal_states.cat.categories.tolist()
print("Terminal states:", terminal_states)

# Optional protocol-aligned evaluation: compare PTK vs CTK using TSI and CBC
cluster_key = "celltype"
rep = "X_pca"

tsi_n_states = max(int(ctk_n_states) if ctk_n_states is not None else 6, 6)
ct_tsi = g.tsi(n_macrostates=tsi_n_states, terminal_states=TERMINAL_STATES, cluster_key=cluster_key)
pt_tsi = g2.tsi(n_macrostates=tsi_n_states, terminal_states=TERMINAL_STATES, cluster_key=cluster_key)
print(f"TSI (CytoTRACEKernel): {ct_tsi:.3f}")
print(f"TSI (PseudotimeKernel): {pt_tsi:.3f}")

ct_rows, pt_rows = [], []
for source, target in STATE_TRANSITIONS:
    ct_vals = np.asarray(ctk.cbc(source=source, target=target, cluster_key=cluster_key, rep=rep))
    pt_vals = np.asarray(pk.cbc(source=source, target=target, cluster_key=cluster_key, rep=rep))
    ct_rows.append(pd.DataFrame({"state_transition": [f"{source}-{target}"] * len(ct_vals), "cbc": ct_vals}))
    pt_rows.append(pd.DataFrame({"state_transition": [f"{source}-{target}"] * len(pt_vals), "cbc": pt_vals}))

ct_cbc = pd.concat(ct_rows, ignore_index=True)
pt_cbc = pd.concat(pt_rows, ignore_index=True)
cbc_ratio = np.log((pt_cbc.groupby("state_transition")["cbc"].mean() + 1) / (ct_cbc.groupby("state_transition")["cbc"].mean() + 1))
display(cbc_ratio.sort_values(ascending=False).to_frame("log_ratio_ptk_vs_ctk"))

In [ ]:
drivers = g2.compute_lineage_drivers(
    lineages=terminal_states,
    return_drivers=True,
)
drivers.head(10)

In [ ]:
for lineage in terminal_states:
    col = f"{lineage}_corr"
    if col in drivers.columns:
        top_genes = drivers[col].sort_values(ascending=False).head(3).index.tolist()
        print(f"\nTop drivers for {lineage}: {top_genes}")
        sc.pl.umap(adata_bm, color=top_genes, ncols=3)

### Step 10: Gene Expression Trends

Model gene expression along lineages using **Generalized Additive Models (GAMs)** and visualize as heatmaps.

In [ ]:
from cellrank.models import GAM

model = GAM(adata_bm)

In [ ]:
cr.pl.gene_trends(
    adata_bm,
    model=model,
    genes=["HBB", "CA1", "MPO"],
    time_key="palantir_pseudotime",
    same_plot=True,
    ncols=3,
    hide_cells=True,
)

In [ ]:
cr.pl.heatmap(
    adata_bm,
    model=model,
    genes=drivers.head(50).index.tolist()[:20],
    time_key="palantir_pseudotime",
    show_fate_probabilities=True,
)

---

## Procedure 3: VelocityKernel (Spermatogenesis)

### About the Dataset

Mouse spermatogenesis scRNA-seq. Cells progress through distinct stages: spermatogonia → spermatocytes → round spermatids → elongated spermatids.

This dataset has spliced/unspliced counts, making it ideal for the **VelocityKernel** which uses RNA velocity to infer transition probabilities.

### Step 11: Load Data and Preprocess

In [ ]:
adata_sp = sc.read_h5ad("data/spermatogenesis/processed/adata.h5ad")
adata_sp.obs["cell_type"] = adata_sp.obs["clusters"]
adata_sp

In [ ]:
# Compute UMAP (not included in the preprocessed data)
sc.tl.umap(adata_sp)
sc.pl.umap(adata_sp, color="clusters", legend_loc="right margin")

### Step 12: Compute RNA Velocity

RNA velocity estimates the **rate of change** of gene expression by modeling splicing kinetics:

$$\frac{du}{dt} = \alpha(t) - \beta u(t)$$
$$\frac{ds}{dt} = \beta u(t) - \gamma s(t)$$

The **velocity** is: $v = \frac{ds}{dt} = \beta u - \gamma s$

scVelo's **dynamical model** fits a full ODE for each gene.

In [ ]:
# The preprocessed data already has PCA, neighbors, and moments computed.
# If starting from raw, you would run:
# scv.pp.filter_and_normalize(adata_sp, min_shared_counts=20, n_top_genes=2000)
# sc.tl.pca(adata_sp)
# sc.pp.neighbors(adata_sp, n_pcs=30, n_neighbors=30)
# scv.pp.moments(adata_sp, n_pcs=None, n_neighbors=None)

print("Layers available:", list(adata_sp.layers.keys()))
print("Shape:", adata_sp.shape)

In [ ]:
# The preprocessed data already has RNA velocity computed.
# If starting from raw, you would run:
# scv.tl.recover_dynamics(adata_sp, n_jobs=8)
print("Velocity layer present:", "velocity" in adata_sp.layers)

In [ ]:
# Velocity is already computed; just build the velocity graph
# If starting from raw: scv.tl.velocity(adata_sp, mode="dynamical")
scv.tl.velocity_graph(adata_sp)

In [ ]:
sp_color = "cell_type" if "cell_type" in adata_sp.obs.columns else "clusters"

scv.pl.velocity_embedding_stream(
    adata_sp,
    basis="umap",
    color=sp_color,
    legend_loc="right margin",
    title="RNA Velocity Streamlines — Spermatogenesis",
)

In [ ]:
# Compute latent time from the dynamical model fit
scv.tl.latent_time(adata_sp)
sc.pl.umap(adata_sp, color="latent_time", color_map="gnuplot", title="Latent Time")

### Step 13: Compute VelocityKernel

The VelocityKernel converts RNA velocity vectors into a cell-cell transition matrix:

$$\tilde{p}_{ij} \propto \exp\left(\frac{\cos\angle(v_i,\, x_j - x_i)}{\sigma}\right)$$

In [ ]:
from cellrank.kernels import VelocityKernel, ConnectivityKernel

vk = VelocityKernel(adata_sp)
vk.compute_transition_matrix()
print(vk)

In [ ]:
ck = ConnectivityKernel(adata_sp)
ck.compute_transition_matrix()

In [ ]:
combined_kernel = 0.8 * vk + 0.2 * ck
print(combined_kernel)

In [ ]:
combined_kernel.plot_projection(basis="umap", color="clusters")

### Step 14: Identify Terminal States and Compute Fate Probabilities

In [ ]:
g3 = GPCCA(combined_kernel)
g3.compute_schur(n_components=20)
g3.plot_spectrum(real_only=True)

In [ ]:
g3.compute_macrostates(n_states=6, cluster_key="clusters")
g3.plot_macrostates(which="all", basis="umap", legend_loc="right margin", s=100)

In [ ]:
g3.plot_coarse_T()

In [ ]:
g3.predict_terminal_states()
g3.plot_macrostates(which="terminal", basis="umap", legend_loc="right margin", s=100)

In [ ]:
import numpy as np
from cellrank._utils._lineage import Lineage

terminal_names = list(g3.terminal_states.cat.categories)
print("Terminal states:", terminal_names)

if len(terminal_names) == 1:
    print("Single terminal state detected; assigning deterministic fate probabilities.")
    fate_probs = Lineage(
        np.ones((adata_sp.n_obs, 1), dtype=float),
        names=terminal_names,
    )
    g3._write_fate_probabilities(fate_probs, params={"solver": "deterministic_single_terminal"})
else:
    try:
        g3.compute_fate_probabilities(solver="gmres", tol=1e-8, preconditioner="ilu")
    except (TypeError, ValueError):
        g3.compute_fate_probabilities(solver="direct")

g3.plot_fate_probabilities(basis="umap", same_plot=False)

### Step 15: Driver Genes and Gene Trends (Spermatogenesis)

In [ ]:
lineages_sp = g3.terminal_states.cat.categories.tolist()

try:
    drivers_sp = g3.compute_lineage_drivers(
        lineages=lineages_sp,
        return_drivers=True,
    )
except RuntimeError as e:
    if "stationary distribution" in str(e):
        g3.compute_eigendecomposition(k=20)
        drivers_sp = g3.compute_lineage_drivers(
            lineages=lineages_sp,
            return_drivers=True,
        )
    else:
        raise

drivers_sp.head(10)

In [ ]:
model_sp = GAM(adata_sp)

cr.pl.gene_trends(
    adata_sp,
    model=model_sp,
    genes=drivers_sp.head(9).index.tolist()[:3],
    time_key="latent_time",
    same_plot=True,
    ncols=3,
    hide_cells=False,
)

In [ ]:
cr.pl.heatmap(
    adata_sp,
    model=model_sp,
    genes=drivers_sp.head(50).index.tolist()[:20],
    time_key="latent_time",
    show_fate_probabilities=True,
)

---

## Procedure 4: RealTimeKernel with Optimal Transport (LARRY)

### About the Dataset

The **LARRY** dataset (Weinreb et al., 2020) profiles mouse hematopoiesis with **lineage tracing** barcodes. Cells are collected at multiple time points (day 2, 4, 6). Since scRNA-seq is destructive, we cannot track individual cells — instead, **optimal transport** probabilistically maps cells across time points.

### Optimal Transport Formulation

$$\pi^* = \arg\min_{\pi \in \Pi(\mu_k, \mu_{k+1})} \sum_{i,j} c(x_i, x_j) \pi_{ij} + \varepsilon \sum_{i,j} \pi_{ij} \log \pi_{ij}$$

The `moscot` package handles OT computation, and CellRank's `RealTimeKernel` converts the coupling into a transition matrix.

### Step 16: Load LARRY Data

In [ ]:
import anndata as ad

adata_larry = ad.read_zarr("data/larry/processed/adata_subsetted_and_processed.zarr.zip")

# Compute UMAP (neighbors graph is already in the data)
sc.tl.umap(adata_larry)
adata_larry

In [ ]:
print("Time points:", sorted(adata_larry.obs["day"].unique().tolist()))
print("\nCells per time point:")
print(adata_larry.obs["day"].value_counts().sort_index())

In [ ]:
sc.pl.umap(adata_larry, color=["day", "cell_type"], ncols=2)

### Step 17: Compute Optimal Transport Coupling with moscot

In [ ]:
from moscot.problems.time import TemporalProblem

tp = TemporalProblem(adata_larry)
tp = tp.prepare(time_key="day")
tp = tp.solve(epsilon=1e-3, tau_a=0.95, tau_b=0.95)

### Step 18: Compute RealTimeKernel from moscot Solution

In [ ]:
from cellrank.kernels import RealTimeKernel

rtk = RealTimeKernel.from_moscot(tp)
rtk.compute_transition_matrix()
print(rtk)

In [ ]:
rtk.plot_projection(basis="umap", color="cell_type")

### Step 19: Identify Terminal States and Fate Probabilities

In [ ]:
g4 = GPCCA(rtk)
g4.compute_schur(n_components=20)
g4.plot_spectrum(real_only=True)

In [ ]:
g4.compute_macrostates(n_states=6, cluster_key="cell_type")
g4.predict_terminal_states()
g4.plot_macrostates(which="terminal", basis="umap", legend_loc="right margin", s=100)

In [ ]:
g4.compute_fate_probabilities()
g4.plot_fate_probabilities(basis="umap", same_plot=False)

### Step 20: Validate with Lineage Tracing Barcodes

A unique advantage of the LARRY dataset: we can **validate** CellRank's inferred fate probabilities against the ground-truth lineage barcodes.

In [ ]:
if "clone_id" in adata_larry.obs.columns:
    print("Clone IDs available for validation")
    print(f"Number of unique clones: {adata_larry.obs['clone_id'].nunique()}")

### Step 21: Driver Genes (LARRY)

In [ ]:
drivers_larry = g4.compute_lineage_drivers(
    lineages=g4.terminal_states.cat.categories.tolist(),
    return_drivers=True,
)
drivers_larry.head(10)

In [ ]:
for lineage in g4.terminal_states.cat.categories.tolist():
    col = f"{lineage}_corr"
    if col in drivers_larry.columns:
        top_genes = drivers_larry[col].sort_values(ascending=False).head(3).index.tolist()
        print(f"\nTop drivers for {lineage}: {top_genes}")
        sc.pl.umap(adata_larry, color=top_genes, ncols=3)

---

## Summary & Key Takeaways

### CellRank 2 Workflow

```
1. Preprocess scRNA-seq data (scanpy/scVelo)
         ↓
2. Choose & compute kernel(s):
   • VelocityKernel   (RNA velocity)
   • PseudotimeKernel (pseudotime ordering)
   • RealTimeKernel   (time-series + OT via moscot)
   • CytoTRACEKernel  (gene count-based potency)
         ↓
3. Optionally combine kernels (weighted sum/product)
         ↓
4. GPCCA estimator:
   • Schur decomposition → eigenvalue spectrum
   • Macrostate identification
   • Terminal / initial state classification
         ↓
5. Compute fate probabilities (absorption probabilities)
         ↓
6. Downstream analysis:
   • Driver gene identification
   • Gene expression trends (GAMs)
   • Fate probability visualization
```

### When to Use Which Kernel?

| Scenario | Recommended Kernel | Example in this Tutorial |
|----------|-------------------|-------------------------|
| No velocity, no time, no pseudotime | `CytoTRACEKernel` | Bone marrow (Procedure 1) |
| Pseudotime available | `PseudotimeKernel` | Bone marrow (Procedure 2) |
| scRNA-seq with splicing info | `VelocityKernel` + `ConnectivityKernel` | Spermatogenesis (Procedure 3) |
| Time-series experiment | `RealTimeKernel` (via moscot) | LARRY (Procedure 4) |
| Multiple signals available | Combine kernels with weighted sum | Procedure 3 (80% Vk + 20% Ck) |

### References

- Lange et al. (2024) *CellRank 2: unified fate mapping in multiview single-cell data.* Nature Methods.
- Weiler et al. (2024) *CellRank 2 Protocol.* Nature Protocols.
- Lange et al. (2022) *CellRank for directed single-cell fate mapping.* Nature Methods.
- Bergen et al. (2020) *Generalizing RNA velocity to transient cell states through dynamical modeling.* Nature Biotechnology.
- Klein et al. (2023) *moscot: Multi-omic single-cell optimal transport.*
- Weinreb et al. (2020) *Lineage tracing on transcriptional landscapes links state to fate during differentiation.* Science.

In [ ]:
import session_info
session_info.show()